# Amazon Bedrock Knowledge Bases로 구현하는 Graph RAG
이 모듈에서는 Amazon Bedrock Knowledge Bases와 Retrieval-Augmented Generation(RAG)을 결합해 그래프 데이터를 더해 정확하고 문맥 있는 설명 가능한 응답을 제공하는 방법을 학습합니다.

다음 단계에서는 Amazon Bedrock Knowledge Bases와 GraphRAG를 함께 사용해 생성형 AI 애플리케이션을 향상시키는 절차를 안내합니다.
이 모듈의 구성은 다음과 같습니다:
1. [Overview](#Overview)
2. [Pre-requisites](#Pre-requisites)
3. [Setup](#Setup)
4. [Create KB with Amazon Neptune as vector](#Create-KB-with-Amazon-Neptune-vector)
5. [Test the Knowledge Graph](#Test-the-Knowledge-Graph)

## Overview
Amazon Bedrock Knowledge Bases는 Amazon Neptune과 연동되는 완전관리형 GraphRAG 기능을 제공합니다. 이 기능은 그래프와 RAG 기법을 결합해 생성형 AI 애플리케이션이 더 정확하고 포괄적인 응답을 제공하도록 돕습니다.

GraphRAG는 Amazon Bedrock Knowledge Bases에 적재된 문서 전반에서 관련 엔터티와 구조적 요소(예: 섹션 제목 등) 간의 관계를 자동으로 식별하고 활용합니다. 이를 통해 여러 문서 청크에 걸친 데이터 연결과 추론이 필요한 상황에서도 더 관련성 높은 응답을 제공할 수 있습니다.

Amazon Bedrock Knowledge Bases는 Amazon Neptune 그래프의 생성과 관리를 자동으로 처리하므로, 그래프 기법 전문 지식이 없어도 엔드유저에게 관련 응답을 제공할 수 있습니다.

GraphRAG를 사용하는 Amazon Bedrock Knowledge Bases의 이점은 다음과 같습니다:

- 관련 엔터티와 문서 섹션의 컨텍스트 정보를 활용해 더 관련성 높은 응답 제공

- 불필요한 정보는 제외하고 데이터 소스의 핵심 콘텐츠를 반영해 더 나은 요약 제공

- 데이터 세트의 다양한 엔터티 간 관계를 이해하고 인용을 제공함으로써 설명 가능한 응답 제공

**참고**: GraphRAG는 Amazon Bedrock Knowledge Bases와 Amazon Neptune Analytics가 모두 제공되는 AWS 리전에서 사용할 수 있습니다.

- 이 모듈에서는 최대 결과 수와 프롬프트 커스터마이징 기능을 활용하기 위해 Anthropic Claude 3 Haiku 모델을 Foundation Model로 사용합니다.

## Pre-requisites
질문에 답하려면 문서를 처리해 Knowledge Base에 저장해야 합니다. 이 노트북에서는 Amazon Bedrock Knowledge Bases를 생성하기 위해 `synthetic dataset for 10K financial reports`를 사용합니다. 

1. 문서(데이터 소스)를 Amazon S3 버킷에 업로드합니다.

## Setup

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### boto3 클라이언트 초기화
노트북 전반에 걸쳐 Knowledge Base 기능을 테스트하기 위해 RetrieveAndGenerate를 사용할 예정입니다.

In [ ]:
import json
import boto3
import pprint
import sys
from botocore.exceptions import ClientError
from botocore.client import Config
import time
import os
import logging

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
    
from utils.knowledge_base import BedrockKnowledgeBase

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
account_id = sts_client.get_caller_identity()['Account']

# Create s3 client
s3_client = boto3.client('s3')

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "anthropic.claude-3-haiku-20240307-v1:0" # we will be using Anthropic Claude 3 Haiku throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'


In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-sample-knowledge-base-{suffix}"
knowledge_base_description = "Graph RAG knowledge base."

bucket_name = f'{knowledge_base_name}-{account_id}'

## Amazon Neptune을 벡터 스토어로 사용하는 Knowledge Base 생성
GraphRAG를 구축하려면 벡터 스토어로 Amazon Neptune Analytics를 선택해야 합니다.
Knowledge Base는 문서 임베딩과 문서 코퍼스에서 추출한 엔터티 및 관계의 그래프 표현을 자동으로 생성해 Amazon Neptune에 저장합니다.

GraphRAG 기반 애플리케이션이 실행되는 동안에도 Knowledge Bases API 작업을 사용해 더욱 포괄적이고 설명 가능한 응답을 제공할 수 있습니다.

In [ ]:
data_sources = [{"type": "S3", "bucket_name": bucket_name}]

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "GRAPH", 
    suffix = f'{suffix}-f',
    vector_store="NEPTUNE_ANALYTICS" # can be OPENSEARCH_SERVERLESS or NEPTUNE_ANALYTICS
)

## Knowledge Base에 적재할 데이터 다운로드
다음 데이터를 사용합니다:

첫 번째 데이터 소스로 로컬 디렉터리에 저장된 합성 데이터
### S3 버킷 데이터 소스에 데이터 업로드

In [ ]:
import os
s3_client = boto3.client('s3')

def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("../synthetic_dataset", bucket_name)

## Ingestion 작업 시작
Knowledge Base와 데이터 소스를 생성한 후 각 데이터 소스에 대한 ingestion 작업을 시작할 수 있습니다. Ingestion 작업 동안 Knowledge Base는 데이터 소스의 문서를 가져와 텍스트를 추출하고, 지정된 청킹 크기에 따라 분할하며, 각 청크의 임베딩을 생성한 뒤 이번 사례에서는 Neptune Analytics에 해당 임베딩을 저장합니다.

참고: 현재는 한 번에 하나의 ingestion 작업만 실행할 수 있습니다.

In [ ]:
# ensure that the kb is available
time.sleep(60)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

## 2.2 Amazon Bedrock Knowledge Bases로 생성된 Knowledge Graph 테스트
사용 사례에 따라 `retrieve` 또는 `retrieve_and_generate` API를 사용해 Amazon Bedrock Knowledge Bases가 생성한 `Neptune Analytics` 그래프를 질의할 수 있습니다.

### Retrieve and Generate API로 Knowledge Base 테스트
먼저 `retrieve_and_generate` API를 사용해 테스트합니다. 이 API는 Amazon Bedrock Knowledge Bases가 생성한 `Neptune Analytics` 그래프를 질의해 검색 결과를 가져오고, 프롬프트와 결합한 뒤 제공된 Foundation Model로 응답을 생성합니다. 

`query = Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

해당 질의의 정답(ground truth QA 쌍 기준)은 다음과 같습니다:

```
According to the consolidated statements of cash flows, in the fiscal year ended December 31, 2019, Octank Financial had:
- Net cash provided by operating activities of $710 million
- Net cash used in investing activities of $240 million
- Net cash provided by financing activities of $350 million
- An overall net increase in cash and cash equivalents of $120 million
```

**참고**: 선택한 모델에 따라 실제 응답은 다를 수 있지만 의미적으로는 유사해야 합니다. 

In [ ]:
query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019?"


In [ ]:
# foundation_model = "amazon.nova-micro-v1:0"

response = bedrock_agent_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)


Retrieve and Generate API를 사용하면 최종 응답을 바로 받게 되므로, 응답에 사용된 다양한 소스를 확인할 수 없습니다. 이제 Retrieve API로 Knowledge Base의 소스 정보를 확인해 보겠습니다.

### Retrieve API로 Knowledge Base 테스트
추가적인 제어가 필요하면 Retrieve API를 사용해 질의와 가장 잘 일치하는 청크를 가져올 수 있습니다. 벡터 스토어가 `Neptune Analytics`이므로, Retrieve API는 그래프를 질의해 관련 청크(검색 결과)를 가져옵니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 응답을 제어할 수 있습니다. API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수, 청크 메타데이터를 제공합니다.

In [ ]:
response_ret = bedrock_agent_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        "text": "How many new positions were opened across Amazon's fulfillment and delivery network?"
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)


## 정리
아래 섹션의 주석을 해제한 뒤 실행해 모든 리소스를 삭제하세요.

In [ ]:
# delete role and policies
print("===============================Deleting Knowledge Base and associated resources==============================\n")
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

<div class="alert alert-block alert-warning">
<b>참고:</b> 비용이 발생하지 않도록 KB, OSS 인덱스 및 관련 IAM 역할과 정책을 반드시 삭제하세요.
</div>